In [1]:
import torch as t
from torch.utils.data import DataLoader
import torchvision as tv

In [2]:
#预处理数据
#oTensor()起到的作用是把PIL.Image或者numpy.narray数据类型转变为torch.FloatTensor类型，shape是C*H*W，数值范围缩小为[0.0, 1.0]。
#0.5和0.5代表均值和方差，均一化；三通道里边3个数
transform = tv.transforms.Compose([tv.transforms.ToTensor(),
                                  tv.transforms.Normalize((0.5,), (0.5,)),
                             ])


In [3]:
#ataLoader是PyTorch中数据读取重要接口，用PyTorch来训练模型基本都会用到该接口，接口将自定义的Dataset根据batch size大小、是否shuffle等封装成一个Batch Size大小的Tensor，用于后面的训练。
train_ts = tv.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_ts = tv.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_dl = DataLoader(train_ts, batch_size=32, shuffle=True, drop_last=False)
test_dl = DataLoader(test_ts, batch_size=64, shuffle=True, drop_last=False)

Using downloaded and verified file: ./data\MNIST\raw\train-images-idx3-ubyte.gz
Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw
Using downloaded and verified file: ./data\MNIST\raw\train-labels-idx1-ubyte.gz
Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw
Using downloaded and verified file: ./data\MNIST\raw\t10k-images-idx3-ubyte.gz
Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw
Using downloaded and verified file: ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz
Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw
Processing...


..\torch\csrc\utils\tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


Done!


In [4]:
#输入层：784个神经元
#隐藏层：100个神经元
#输出层：10个神经元

model = t.nn.Sequential(
   t.nn.Linear(784, 100),
   t.nn.ReLU(),
   t.nn.Linear(100, 10),
   t.nn.LogSoftmax(dim=1)
)

In [5]:
#定义损失函数--均方误差损失与优化函数
loss_fn = t.nn.NLLLoss(reduction="mean")
optimizer = t.optim.Adam(model.parameters(), lr=1e-3)

In [6]:
#训练,view相当于reshape;
for s in range(5):
   print("run in step : %d"%s)
   for i, (x_train, y_train) in enumerate(train_dl):
       x_train = x_train.view(x_train.shape[0], -1)
       y_pred = model(x_train)
       train_loss = loss_fn(y_pred, y_train)
       if (i + 1) % 100 == 0:
           print(i + 1, train_loss.item())
       model.zero_grad()
       train_loss.backward()
       optimizer.step()

run in step : 0
100 0.4118703603744507
200 0.5581432580947876
300 0.24970321357250214
400 0.17116037011146545
500 0.2641041874885559
600 0.38658925890922546
700 0.12484250217676163
800 0.4279106557369232
900 0.1375202238559723
1000 0.22412562370300293
1100 0.3093990683555603
1200 0.1526477038860321
1300 0.2435932159423828
1400 0.2934589684009552
1500 0.564362645149231
1600 0.15247757732868195
1700 0.44141441583633423
1800 0.08823318779468536
run in step : 1
100 0.22151604294776917
200 0.2081320434808731
300 0.18304476141929626
400 0.13293762505054474
500 0.24954016506671906
600 0.23794038593769073
700 0.16781719028949738
800 0.12840025126934052
900 0.0952371209859848
1000 0.23072166740894318
1100 0.46371299028396606
1200 0.30126112699508667
1300 0.381339967250824
1400 0.09298519045114517
1500 0.11991730332374573
1600 0.4551672041416168
1700 0.039841894060373306
1800 0.16063179075717926
run in step : 2
100 0.3408887982368469
200 0.296337366104126
300 0.13817785680294037
400 0.2089039534

In [7]:
#预测
total = 0;
correct_count = 0
for test_images, test_labels in test_dl:
   for i in range(len(test_labels)):
       image = test_images[i].view(1, 784)#reshape
       with t.no_grad():#预测时不要计算梯度
           pred_labels = model(image)
       plabels = t.exp(pred_labels)#实现以e为底的指数
       probs = list(plabels.numpy()[0])
       pred_label = probs.index(max(probs))
       true_label = test_labels.numpy()[i]
       if pred_label == true_label:
           correct_count += 1
       total += 1
print("total acc : %.2f\n"%(correct_count / total))
t.save(model, './nn_mnist_model.pt')

total acc : 0.96



In [16]:
x = t.FloatTensor([1,2,1])
x.exp()

tensor([2.7183, 7.3891, 2.7183])